# AGI Node.js SDK - Quickstart
This notebook demonstrates how to use the AGI Node.js SDK to create and control remote browser agents.

## Setup
Import the SDK and create a client.

In [ ]:
import { AGIClient } from 'agi';

const client = new AGIClient({ apiKey: process.env.AGI_API_KEY });
console.log('Connected to AGI API');

## Create a Session
Create a remote browser session using the `agi-0` agent.

In [ ]:
const session = await client.sessions.create('agi-0');
console.log(`Session ID: ${session.sessionId}`);
console.log(`Status: ${session.status}`);

## Send a Task
Send a task to the agent and stream the results in real-time.

In [ ]:
await client.sessions.sendMessage(session.sessionId, 'What is the current time? Search Google to find out.');

for await (const event of client.sessions.streamEvents(session.sessionId)) {
    const eventType = event.event;
    let content = '';
    
    if (event.data) {
        const data = event.data;
        if (typeof data === 'object') {
            content = data.content || data.message || data.text || JSON.stringify(data);
        } else {
            content = String(data);
        }
    }
    
    switch (eventType) {
        case 'thought':
            console.log(`Thought: ${content.slice(0, 100)}`);
            break;
        case 'step':
            console.log(`Step: ${content.slice(0, 100)}`);
            break;
        case 'done':
            console.log(`Done: ${content.slice(0, 200)}`);
            break;
        case 'error':
            console.log(`Error: ${content}`);
            break;
        case 'question':
            console.log(`Agent asks: ${content}`);
            break;
    }
    
    if (eventType === 'done' || eventType === 'error') break;
}

## Take a Screenshot
Capture the current state of the remote browser.

In [ ]:
const screenshot = await client.sessions.screenshot(session.sessionId);
let imgData = screenshot.screenshot;
if (imgData.includes(',')) {
    imgData = imgData.split(',')[1];
}
// Display as base64 image
const buffer = Buffer.from(imgData, 'base64');
console.log(`Screenshot captured (${buffer.length} bytes)`);

## Session Control
Pause, resume, or cancel the agent.

In [ ]:
await client.sessions.pause(session.sessionId);
console.log('Session paused');

await client.sessions.resume(session.sessionId);
console.log('Session resumed');

## Cleanup
Delete the session when done.

In [ ]:
await client.sessions.delete(session.sessionId);
console.log('Session deleted');

## Using the High-Level API
The recommended way - handles cleanup automatically.

In [ ]:
{
    await using session = client.session('agi-0');
    const result = await session.runTask('Search for the latest Node.js LTS version');
    console.log(`Result: ${result.data}`);
    console.log(`Duration: ${result.metadata.duration}s`);
    console.log(`Steps: ${result.metadata.steps}`);
}